In [114]:
import os
import requests
import jwt
import time
import sqlite3
import json

def save_to_file(data, file_path):
    with open(file_path, 'w') as file:
        # file.write(data)
        json.dump(data, file)

def load_from_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def generate_jwt_token(client_id, client_secret):
    # Replace with the appropriate token endpoint URL for your OAuth server
    token_endpoint = "https://web.arbeitsagentur.de/weiterbildungssuche/suche"

    now = int(time.time())
    payload = {
        "iss": client_id,
        "sub": client_id,
        "aud": token_endpoint,
        "iat": now,
        "exp": now + 3600,  # Token expires in 1 hour
    }

    # Sign the payload with your client secret to generate the JWT
    jwt_token = jwt.encode(payload, client_secret, algorithm='HS256')

    return jwt_token

def get_access_token(client_id, client_secret):
    # Generate the JWT token
    jwt_token = generate_jwt_token(client_id, client_secret)

    # Request the access token using the JWT token
    token_endpoint = "https://rest.arbeitsagentur.de/oauth/gettoken_cc"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    response = requests.post(token_endpoint, headers=headers, data=data)

    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to get access token. Status code: {response.status_code}")

def get_api_data(api_url, access_token):
    headers = {
        "Authorization": f"Bearer {access_token}",
    }

    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if data:  # Check if data is not empty
            return data
        else:
            print("No more data")
    else:
        raise Exception(f"Failed to fetch data from API. Status code: {response.status_code}")

def create_table(db_connection):
    cursor = db_connection.cursor()
    # Create a table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS weiterbildung_data
                            (
                                angebot_id INT,
                                angebot_titel TEXT NULL,
                                angebot_inhalt TEXT NULL,
                                angebot_abschlussart TEXT NULL,
                                angebot_abschlussbezeichnung TEXT NULL,
                                angebot_foerderung TEXT NULL,
                                angebot_zugang TEXT NULL,
                                angebot_anrechnung TEXT NULL,
                                angebot_berechtigungen TEXT NULL,
                                angebot_zusatzqualifikationen TEXT NULL,
                                angebot_link TEXT NULL,
                                angebot_zielgruppe TEXT NULL,
                                angebot_bildungsanbieter_id INT NULL,
                                angebot_bildungsanbieter_name TEXT NULL,
                                bildungsanbieter_adresse_id INT NULL,
                                bildungsanbieter_adresse_bezeichnung TEXT NULL,
                                bildungsanbieter_adresse_strasse TEXT NULL,
                                bildungsanbieter_adresse_hinweise TEXT NULL,
                                bildungsanbieter_ortStrasse_id INT NULL,
                                bildungsanbieter_ortStrasse_plz TEXT NULL,
                                bildungsanbieter_ortStrasse_name TEXT NULL,
                                bildungsanbieter_ortStrasse_land_id INT NULL,
                                bildungsanbieter_ortStrasse_land_name TEXT NULL,
                                bildungsanbieter_ortStrasse_land_laenderCode TEXT NULL,
                                bildungsanbieter_ortStrasse_land_bundeslandCode TEXT NULL,
                                bildungsanbieter_ortStrasse_land_code TEXT NULL,
                                bildungsanbieter_koordinaten_lat REAL NULL,
                                bildungsanbieter_koordinaten_lon REAL NULL,
                                adresse_id INT NULL,
                                adresse_bezeichnung TEXT NULL,
                                adresse_strasse TEXT NULL,
                                adresse_hinweise TEXT NULL,
                                ortStrasse_id INT NULL,
                                ortStrasse_plz TEXT NULL,
                                ortStrasse_name TEXT NULL,
                                ortStrasse_land_id INT NULL,
                                ortStrasse_land_name TEXT NULL,
                                ortStrasse_land_laenderCode TEXT NULL,
                                ortStrasse_land_bundeslandCode TEXT NULL,
                                ortStrasse_land_code TEXT NULL,
                                koordinaten_lat REAL NULL,
                                koordinaten_lon REAL NULL,
                                bildungsart_id INT NULL,
                                bildungsart_bezeichnung TEXT NULL
                                )
                ''')
    db_connection.commit()

In [115]:
def insert_data_into_db(data, db_connection):
    cursor = db_connection.cursor()
    for entry in data:
        unterrichtsform = entry["unterrichtsform"]
        unterrichtszeit = entry["unterrichtszeit"]
        dauer = entry["dauer"]
        angebot = entry["angebot"]
        adresse = entry["adresse"]
        
        if(adresse != None):
            bildungsanbieter_address = angebot["bildungsanbieter"]["adresse"]
            cursor.execute('''INSERT INTO weiterbildung_data (
                    angebot_id, angebot_titel, angebot_inhalt, angebot_abschlussart, angebot_abschlussbezeichnung,
                    angebot_foerderung, angebot_zugang, angebot_anrechnung,
                    angebot_berechtigungen, angebot_zusatzqualifikationen, angebot_link,
                    angebot_zielgruppe, angebot_bildungsanbieter_id,
                    angebot_bildungsanbieter_name, bildungsanbieter_adresse_id, bildungsanbieter_adresse_bezeichnung,
                    bildungsanbieter_adresse_strasse, bildungsanbieter_adresse_hinweise, bildungsanbieter_ortStrasse_id,
                    bildungsanbieter_ortStrasse_plz, bildungsanbieter_ortStrasse_name,
                    bildungsanbieter_ortStrasse_land_id, bildungsanbieter_ortStrasse_land_name,
                    bildungsanbieter_ortStrasse_land_laenderCode, bildungsanbieter_ortStrasse_land_bundeslandCode,
                    bildungsanbieter_ortStrasse_land_code, bildungsanbieter_koordinaten_lat,
                    bildungsanbieter_koordinaten_lon,adresse_id,adresse_bezeichnung, 
                    adresse_strasse, adresse_hinweise, ortStrasse_id, ortStrasse_plz,
                    ortStrasse_name, ortStrasse_land_id, ortStrasse_land_name, ortStrasse_land_laenderCode,
                    ortStrasse_land_bundeslandCode, ortStrasse_land_code, koordinaten_lat,
                    koordinaten_lon, bildungsart_id,
                    bildungsart_bezeichnung
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                angebot["id"], angebot["titel"], angebot["inhalt"], angebot["abschlussart"],
                angebot["abschlussbezeichnung"], angebot["foerderung"], angebot["zugang"], angebot["anrechnung"],
                angebot["berechtigungen"], angebot["zusatzqualifikationen"], angebot["link"], angebot["zielgruppe"],
                angebot["bildungsanbieter"]["id"], angebot["bildungsanbieter"]["name"], 
                bildungsanbieter_address["id"], bildungsanbieter_address["bezeichnung"],bildungsanbieter_address["strasse"],
                bildungsanbieter_address["hinweise"], bildungsanbieter_address["ortStrasse"]["id"],
                bildungsanbieter_address["ortStrasse"]["plz"], bildungsanbieter_address["ortStrasse"]["name"],
                bildungsanbieter_address["ortStrasse"]["land"]["id"], bildungsanbieter_address["ortStrasse"]["land"]["name"], 
                bildungsanbieter_address["ortStrasse"]["land"]["laenderCode"],bildungsanbieter_address["ortStrasse"]["land"]["bundeslandCode"],
                bildungsanbieter_address["ortStrasse"]["land"]["code"], bildungsanbieter_address["koordinaten"]["lat"],
                bildungsanbieter_address["koordinaten"]["lon"],adresse["id"],adresse["bezeichnung"],adresse["strasse"],
                adresse["hinweise"],adresse["ortStrasse"]["id"],adresse["ortStrasse"]["plz"],
                adresse["ortStrasse"]["name"],adresse["ortStrasse"]["land"]["id"],adresse["ortStrasse"]["land"]["name"], adresse["ortStrasse"]["land"]["laenderCode"],
                adresse["ortStrasse"]["land"]["bundeslandCode"],adresse["ortStrasse"]["land"]["code"], adresse["koordinaten"]["lat"],
                adresse["koordinaten"]["lon"],angebot["bildungsart"]["id"], angebot["bildungsart"]["bezeichnung"]
            ))
        
        # # Insert data into the table
        # cursor.execute('''INSERT INTO weiterbildung_data (
        #             angebot_id, angebot_titel, angebot_inhalt, angebot_abschlussart, angebot_abschlussbezeichnung,
        #             angebot_foerderung, angebot_zugang, angebot_anrechnung,
        #             angebot_berechtigungen, angebot_zusatzqualifikationen, angebot_link,
        #             angebot_zielgruppe
        #         ) VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        #     ''', (
        #         angebot["id"], angebot["titel"], angebot["inhalt"], angebot["abschlussart"],
        #         angebot["abschlussbezeichnung"], angebot["foerderung"], angebot["zugang"], angebot["anrechnung"],
        #         angebot["berechtigungen"], angebot["zusatzqualifikationen"], angebot["link"], angebot["zielgruppe"]
        #     ))
        
        # if(angebot["bildungsart"]):
        #     cursor.execute('''INSERT INTO weiterbildung_data (bildungsart_id,
        #             bildungsart_bezeichnung) VALUES (?, ?)''', (angebot["bildungsart"]["id"], angebot["bildungsart"]["bezeichnung"]))
        # if(entry["adresse"]):
        #     adresse = entry["adresse"]
        #     cursor.execute('''INSERT INTO weiterbildung_data (adresse_id,adresse_bezeichnung, 
        #             adresse_strasse, adresse_hinweise) VALUES (?, ?, ?, ?)''', (adresse["id"],adresse["bezeichnung"],adresse["strasse"],
        #             adresse["hinweise"]))
        #     if(adresse["koordinaten"]):
        #         cursor.execute('''INSERT INTO weiterbildung_data (koordinaten_lat,
        #                 koordinaten_lon) VALUES (?, ?)''', (adresse["koordinaten"]["lat"],
        #             adresse["koordinaten"]["lon"]))
        
        #     if(adresse["ortStrasse"]):
        #         ortStrasse = adresse["ortStrasse"]
        #         cursor.execute('''INSERT INTO weiterbildung_data (ortStrasse_id, ortStrasse_plz,
        #                 ortStrasse_name) VALUES (?, ?, ?)''', (ortStrasse["id"],ortStrasse["plz"],
        #             ortStrasse["name"]))
        #         if(ortStrasse["land"]):
        #             land = ortStrasse["land"]
        #             cursor.execute('''INSERT INTO weiterbildung_data (ortStrasse_land_id, ortStrasse_land_name, ortStrasse_land_laenderCode,
        #                 ortStrasse_land_bundeslandCode, ortStrasse_land_code) VALUES (?, ?, ?, ?, ?)''', (
        #                     land["id"], land["name"], land["laenderCode"], land["bundeslandCode"], land["code"]))
    
    
        # if(angebot["bildungsanbieter"]):
        #     bildungsanbieter = angebot["bildungsanbieter"]
        #     cursor.execute('''INSERT INTO weiterbildung_data (angebot_bildungsanbieter_id,
        #             angebot_bildungsanbieter_name) VALUES (?, ?)''', (bildungsanbieter["id"], bildungsanbieter["name"]))
        #     if(bildungsanbieter["adresse"]):
        #         bildungsanbieter_address = bildungsanbieter["adresse"]
        #         cursor.execute('''INSERT INTO weiterbildung_data ( bildungsanbieter_adresse_id, bildungsanbieter_adresse_bezeichnung,
        #             bildungsanbieter_adresse_strasse, bildungsanbieter_adresse_hinweise ) VALUES (?, ?, ?, ?)''', 
        #             (bildungsanbieter_address["id"], bildungsanbieter_address["bezeichnung"],
        #             bildungsanbieter_address["strasse"], bildungsanbieter_address["hinweise"]))
        #         if(bildungsanbieter_address["ortStrasse"]):
        #             bildungsanbieter_ortStrasse = bildungsanbieter_address["ortStrasse"]
        #             cursor.execute('''INSERT INTO weiterbildung_data ( bildungsanbieter_ortStrasse_id,
        #                 bildungsanbieter_ortStrasse_plz, bildungsanbieter_ortStrasse_name ) VALUES (?, ?, ?)''', (bildungsanbieter_ortStrasse["id"],
        #                 bildungsanbieter_ortStrasse["plz"], bildungsanbieter_ortStrasse["name"]))
        #             if(bildungsanbieter_ortStrasse["land"]):
        #                 bildungsanbieter_ortStrasse_land = bildungsanbieter_ortStrasse["land"]
        #                 cursor.execute('''INSERT INTO weiterbildung_data ( bildungsanbieter_ortStrasse_land_id, bildungsanbieter_ortStrasse_land_name,
        #                     bildungsanbieter_ortStrasse_land_laenderCode, bildungsanbieter_ortStrasse_land_bundeslandCode,
        #                     bildungsanbieter_ortStrasse_land_code ) VALUES (?, ?, ?, ?, ?)''', (bildungsanbieter_ortStrasse_land["id"], bildungsanbieter_ortStrasse_land["name"], 
        #                     bildungsanbieter_ortStrasse_land["laenderCode"],bildungsanbieter_ortStrasse_land["bundeslandCode"],
        #                     bildungsanbieter_ortStrasse_land["code"]))
        #         if(bildungsanbieter_address["koordinaten"]):
        #             bildungsanbieter_koordinaten = bildungsanbieter_address["koordinaten"]
        #             cursor.execute('''INSERT INTO weiterbildung_data ( bildungsanbieter_koordinaten_lat,
        #                 bildungsanbieter_koordinaten_lon ) VALUES (?, ?)''',(bildungsanbieter_koordinaten["lat"],
        #                 bildungsanbieter_koordinaten["lon"]))
    db_connection.commit()
    print("insert data")

In [116]:
def main():
    api_url = "https://rest.arbeitsagentur.de/infosysbub/wbsuche/pc/v1/bildungsangebot"  # Replace with the API endpoint URL
    data_file = "data_json.txt"
    client_id = "38053956-6618-4953-b670-b4ae7a2360b1"  # Replace with your OAuth client ID
    client_secret = "c385073c-3b97-42a9-b916-08fd8a5d1795"  # Replace with your OAuth client secret

    # Get the access token using client credentials flow with JWTs
    access_token = get_access_token(client_id, client_secret)

    try:
        # Open a connection
        db_connection = sqlite3.connect('weiterbildung_analysis.db')
        
        # Create the table if not exists
        create_table(db_connection)

        # Pagination: Fetch all data from the API using multiple requests
        all_data = []
        page = 0
        totalPages = 2
        while page < totalPages:
            paginated_api_url = f"{api_url}?page={page}&size=20"
            print(paginated_api_url)
            api_data = get_api_data(paginated_api_url, access_token)
            totalPages = api_data["page"]["totalPages"]
            # save_to_file(api_data, data_file)
            # Insert data into the database
            insert_data_into_db(list(api_data["_embedded"]["termine"]), db_connection)
            # all_data.extend(api_data["_embedded"]["termine"])
            page += 1
        
        
        
        # # Insert data into the database
        # insert_data_into_db(all_data, db_connection)

    except Exception as e:
        print(f"Error: {e}")
        db_connection.rollback()  # Rollback changes in case of an error
    
    finally:
        # Close the connection
        db_connection.close()
        print("Data fetched and saved to SQLite database successfully.")


if __name__ == "__main__":
    main()

https://rest.arbeitsagentur.de/infosysbub/wbsuche/pc/v1/bildungsangebot?page=0&size=20
Error: unsupported operand type(s) for &: 'dict' and 'dict'
Data fetched and saved to SQLite database successfully.
